# 🔮 Temporal Fusion Transformer (TFT)

**Phase 4, Step 3: Advanced Transformers for Time Series**

---

## 🎯 Объединяем все знания!

### Что мы изучили до сих пор:

**Phase 3: Temporal Data & RNN**
- ✅ Classical Time Series (ARIMA, SARIMA, Prophet)
- ✅ RNN/LSTM/GRU для sequences
- ✅ Attention & Seq2Seq
- ❌ Но: short-term forecasting, univariate/simple multivariate

**Phase 4 Step 1: Self-Attention & Transformers**
- ✅ Scaled Dot-Product Attention
- ✅ Multi-Head Attention
- ✅ Transformer Encoder
- ❌ Но: применяли к табличным данным, не к time series

**Phase 4 Step 2: TabTransformer**
- ✅ Contextual embeddings для categorical features
- ✅ Attention между features
- ✅ Large dataset (48k samples)
- ❌ Но: static data, не temporal

---

## 🚀 Enter Temporal Fusion Transformer (2020)

**"Temporal Fusion Transformers for Interpretable Multi-horizon Time Series Forecasting"**  
(Lim et al., Google Research)

**Ключевая идея:** Объединить лучшее из всех миров!

```
TFT = RNN/LSTM + Transformers + Variable Selection + Multi-Horizon
```

---

## 📐 Что делает TFT уникальным?

### 1. Multi-Horizon Forecasting

**Problem:** Предсказать не 1 step, а несколько (например, 24 часа вперед)

**Traditional approaches:**
- Recursive: predict t+1, use it to predict t+2, etc. (error accumulation!)
- Direct: train separate models for each horizon (expensive!)

**TFT solution:**
- Single model predicts all horizons simultaneously
- Quantile forecasting (uncertainty estimates)
- No error accumulation

---

### 2. Mixed Data Types

**Time series часто имеют:**
- Static covariates: category, location (не меняются во времени)
- Known future inputs: day_of_week, hour, holidays (известны заранее)
- Unknown future inputs: weather (неизвестны, только historical)
- Target: что предсказываем

**Пример: Electricity Demand Forecasting**
```
Static:     building_id, region
Known:      hour, day_of_week, is_holiday, temperature_forecast
Unknown:    actual_temperature, humidity (только historical)
Target:     electricity_consumption
```

**TFT handles all types!**

---

### 3. Variable Selection Networks

**Problem:** Не все features важны для каждого prediction

**Solution:** Gated Residual Networks (GRN) для automatic feature selection
- Модель сама выбирает, какие features использовать
- Interpretability: можно посмотреть, какие features важны

---

### 4. Interpretable Multi-Head Attention

**Problem:** Black box predictions

**Solution:**
- Self-Attention показывает, на какие timesteps модель смотрит
- Variable importance weights
- Quantile predictions (uncertainty)

**Result:** Можно объяснить каждое предсказание!

---

## 🏗️ TFT Architecture Overview

```
Inputs: [Static, Known Past, Known Future, Unknown Past]
   ↓
┌─────────────────────────────────────────┐
│ 1. Variable Selection Networks          │
│    - Select relevant features            │
│    - Gated Residual Networks (GRN)      │
└──────────────────┬──────────────────────┘
                   ↓
┌─────────────────────────────────────────┐
│ 2. LSTM Encoder-Decoder                 │
│    - Process past sequence               │
│    - Generate context for future        │
└──────────────────┬──────────────────────┘
                   ↓
┌─────────────────────────────────────────┐
│ 3. Multi-Head Attention                 │
│    - Long-range dependencies            │
│    - Interpretable attention weights    │
└──────────────────┬──────────────────────┘
                   ↓
┌─────────────────────────────────────────┐
│ 4. Gated Linear Unit + Layer Norm       │
│    - Residual connections                │
│    - Skip connections                    │
└──────────────────┬──────────────────────┘
                   ↓
┌─────────────────────────────────────────┐
│ 5. Output: Multi-Horizon Predictions    │
│    - Quantile forecasts (P10, P50, P90) │
│    - All horizons simultaneously         │
└─────────────────────────────────────────┘
```

---

## 📊 Use Cases

**TFT применяется для:**
- ⚡ Electricity load forecasting
- 🌤️ Weather prediction
- 💰 Financial market forecasting
- 🚗 Traffic prediction
- 🏥 Hospital resource planning
- 📦 Demand forecasting (retail/supply chain)

**Требования:**
- Multi-step ahead predictions
- Multiple time series (panel data)
- Mixed categorical + numerical features
- Interpretability важна

---

## 🎯 Что мы реализуем

В этом notebook:

### 1. Dataset: Electricity Consumption (Simplified)
- Синтетические данные для демонстрации
- ~10,000 timesteps, multiple households
- Categorical: household_id, day_of_week
- Numerical: hour, temperature
- Target: electricity_consumption

### 2. Simplified TFT Implementation
- Variable Selection Networks (GRN)
- LSTM Encoder
- Multi-Head Attention
- Multi-horizon output (predict 24 hours)

### 3. Training & Evaluation
- Quantile Loss (P10, P50, P90)
- Multi-horizon metrics
- Attention visualization
- Variable importance

### 4. Comparison
- LSTM baseline
- Prophet (from Phase 3)
- TFT (full power)

---


## 💻 Часть 1: Setup и Dataset

### 1.1 Импорт библиотек

In [ ]:
# Базовые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Sklearn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Math
import math

# Настройки
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("\n✅ Все библиотеки загружены")

### 1.2 Создание Synthetic Electricity Dataset

**Создаем реалистичные данные:**
- 10 households (multiple time series)
- 365 days × 24 hours = 8,760 timesteps per household
- Total: 87,600 data points

**Features:**
- Static: household_id
- Known future: hour, day_of_week, is_weekend
- Unknown (historical only): temperature
- Target: electricity_consumption (kWh)

**Patterns:**
- Daily seasonality (peak during evening)
- Weekly seasonality (weekend vs weekday)
- Temperature dependency
- Random household baseline

In [ ]:
def create_electricity_dataset(n_households=10, n_days=365):
    """
    Create synthetic electricity consumption dataset
    """
    print(f"Creating dataset: {n_households} households, {n_days} days")
    
    # Generate timestamps
    start_date = datetime(2023, 1, 1)
    timestamps = [start_date + timedelta(hours=h) for h in range(n_days * 24)]
    
    data_list = []
    
    for household_id in range(n_households):
        # Random household baseline (some consume more than others)
        baseline = np.random.uniform(2, 5)
        
        for i, ts in enumerate(timestamps):
            hour = ts.hour
            day_of_week = ts.weekday()
            is_weekend = 1 if day_of_week >= 5 else 0
            day_of_year = ts.timetuple().tm_yday
            
            # Temperature (seasonal + daily variation)
            temp = 15 + 10 * np.sin(2 * np.pi * day_of_year / 365) + \
                   3 * np.sin(2 * np.pi * hour / 24) + \
                   np.random.normal(0, 2)
            
            # Electricity consumption pattern
            # 1. Baseline
            consumption = baseline
            
            # 2. Daily pattern (peak in evening 18-22h)
            if 6 <= hour <= 8:  # Morning peak
                consumption += 2
            elif 18 <= hour <= 22:  # Evening peak
                consumption += 4
            elif 0 <= hour <= 6:  # Night low
                consumption -= 1
            
            # 3. Weekend effect (more consumption during day)
            if is_weekend and 10 <= hour <= 20:
                consumption += 1.5
            
            # 4. Temperature effect (heating/cooling)
            if temp < 10:  # Heating
                consumption += (10 - temp) * 0.2
            elif temp > 25:  # Cooling
                consumption += (temp - 25) * 0.3
            
            # 5. Random noise
            consumption += np.random.normal(0, 0.5)
            
            # 6. Ensure positive
            consumption = max(0, consumption)
            
            data_list.append({
                'timestamp': ts,
                'household_id': household_id,
                'hour': hour,
                'day_of_week': day_of_week,
                'is_weekend': is_weekend,
                'temperature': temp,
                'consumption': consumption
            })
    
    df = pd.DataFrame(data_list)
    print(f"✅ Dataset created: {len(df):,} rows")
    return df

# Create dataset
df = create_electricity_dataset(n_households=10, n_days=365)

print(f"\nDataset shape: {df.shape}")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\nFirst few rows:")
df.head()

### 1.3 Exploratory Data Analysis

In [ ]:
# Basic statistics
print("Dataset Statistics:")
print("="*60)
print(f"Total samples: {len(df):,}")
print(f"Number of households: {df['household_id'].nunique()}")
print(f"Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\nConsumption statistics (kWh):")
print(df['consumption'].describe())
print(f"\nTemperature statistics (°C):")
print(df['temperature'].describe())
print("="*60)

In [ ]:
# Visualize consumption patterns
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Time series for one household
household_0 = df[df['household_id'] == 0].copy()
axes[0, 0].plot(household_0['timestamp'], household_0['consumption'], 
               linewidth=0.5, alpha=0.7)
axes[0, 0].set_title('Electricity Consumption: Household 0 (1 year)', 
                     fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Consumption (kWh)')
axes[0, 0].grid(alpha=0.3)

# 2. Average by hour of day
hourly_avg = df.groupby('hour')['consumption'].mean()
axes[0, 1].bar(hourly_avg.index, hourly_avg.values, color='steelblue', alpha=0.7)
axes[0, 1].set_title('Average Consumption by Hour of Day', 
                     fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Hour')
axes[0, 1].set_ylabel('Avg Consumption (kWh)')
axes[0, 1].grid(alpha=0.3, axis='y')

# 3. Average by day of week
dow_avg = df.groupby('day_of_week')['consumption'].mean()
dow_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1, 0].bar(dow_names, dow_avg.values, color='orange', alpha=0.7)
axes[1, 0].set_title('Average Consumption by Day of Week', 
                     fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Day of Week')
axes[1, 0].set_ylabel('Avg Consumption (kWh)')
axes[1, 0].grid(alpha=0.3, axis='y')

# 4. Consumption vs Temperature
axes[1, 1].hexbin(df['temperature'], df['consumption'], gridsize=30, cmap='YlOrRd')
axes[1, 1].set_title('Consumption vs Temperature', 
                     fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Temperature (°C)')
axes[1, 1].set_ylabel('Consumption (kWh)')

plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("  - Clear daily pattern: peaks at morning (6-8h) and evening (18-22h)")
print("  - Weekend effect: slightly higher consumption")
print("  - Temperature dependency: U-shaped (heating + cooling)")
print("  - Multiple seasonalities: hourly, daily, weekly")

In [ ]:
# Visualize one week in detail
one_week = household_0.iloc[:24*7].copy()

fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

# Consumption
axes[0].plot(one_week['timestamp'], one_week['consumption'], 
            linewidth=2, marker='o', markersize=3, label='Consumption')
axes[0].set_title('One Week Detail: Household 0', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Consumption (kWh)', fontsize=12)
axes[0].legend()
axes[0].grid(alpha=0.3)

# Temperature
axes[1].plot(one_week['timestamp'], one_week['temperature'], 
            linewidth=2, marker='o', markersize=3, color='orange', label='Temperature')
axes[1].set_xlabel('Time', fontsize=12)
axes[1].set_ylabel('Temperature (°C)', fontsize=12)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Weekly pattern visible:")
print("  - Daily cycles clear")
print("  - Temperature correlation")
print("  - Perfect for multi-horizon forecasting!")

---

## 🔧 Часть 2: Data Preparation для TFT

### 2.1 Sliding Window Approach

**TFT требует:**
- **encoder_length**: сколько historical timesteps использовать (например, 168 = 7 days)
- **decoder_length**: сколько future timesteps предсказать (например, 24 = 1 day)

**Пример:**
```
encoder_length = 168 (7 days × 24 hours)
decoder_length = 24  (1 day)

Historical: [t-168, ..., t-1]  → predict Future: [t, t+1, ..., t+23]
```

In [ ]:
# Hyperparameters
ENCODER_LENGTH = 168  # 7 days lookback
DECODER_LENGTH = 24   # 1 day prediction

print(f"TFT Window Configuration:")
print(f"  Encoder length: {ENCODER_LENGTH} hours ({ENCODER_LENGTH//24} days)")
print(f"  Decoder length: {DECODER_LENGTH} hours ({DECODER_LENGTH//24} day)")
print(f"  Total window: {ENCODER_LENGTH + DECODER_LENGTH} hours")

In [ ]:
def create_tft_dataset(df, encoder_length, decoder_length):
    """
    Create TFT dataset with sliding windows
    
    Returns:
        sequences: list of dicts with encoder/decoder data
    """
    sequences = []
    
    # Process each household separately
    for household_id in df['household_id'].unique():
        household_df = df[df['household_id'] == household_id].sort_values('timestamp').reset_index(drop=True)
        
        total_length = encoder_length + decoder_length
        
        for i in range(len(household_df) - total_length + 1):
            # Encoder data (historical)
            encoder_df = household_df.iloc[i:i+encoder_length]
            
            # Decoder data (future to predict)
            decoder_df = household_df.iloc[i+encoder_length:i+total_length]
            
            sequence = {
                # Static features (same for all timesteps)
                'household_id': household_id,
                
                # Encoder inputs (historical)
                'encoder_hour': encoder_df['hour'].values,
                'encoder_dow': encoder_df['day_of_week'].values,
                'encoder_weekend': encoder_df['is_weekend'].values,
                'encoder_temp': encoder_df['temperature'].values,
                'encoder_consumption': encoder_df['consumption'].values,
                
                # Decoder inputs (future known)
                'decoder_hour': decoder_df['hour'].values,
                'decoder_dow': decoder_df['day_of_week'].values,
                'decoder_weekend': decoder_df['is_weekend'].values,
                
                # Target (what to predict)
                'target': decoder_df['consumption'].values
            }
            
            sequences.append(sequence)
    
    return sequences

print("Creating TFT dataset...")
sequences = create_tft_dataset(df, ENCODER_LENGTH, DECODER_LENGTH)

print(f"\n✅ Created {len(sequences):,} sequences")
print(f"\nExample sequence keys: {list(sequences[0].keys())}")
print(f"\nExample shapes:")
for key, val in sequences[0].items():
    if isinstance(val, np.ndarray):
        print(f"  {key:25s}: {val.shape}")
    else:
        print(f"  {key:25s}: {val}")

In [ ]:
# Train/Validation/Test Split
# Split by time (not random!) for time series

n_sequences = len(sequences)
train_size = int(0.7 * n_sequences)
val_size = int(0.15 * n_sequences)

train_sequences = sequences[:train_size]
val_sequences = sequences[train_size:train_size+val_size]
test_sequences = sequences[train_size+val_size:]

print(f"Dataset split (temporal):")
print(f"  Train: {len(train_sequences):,} sequences ({len(train_sequences)/n_sequences*100:.1f}%)")
print(f"  Val:   {len(val_sequences):,} sequences ({len(val_sequences)/n_sequences*100:.1f}%)")
print(f"  Test:  {len(test_sequences):,} sequences ({len(test_sequences)/n_sequences*100:.1f}%)")
print(f"  Total: {n_sequences:,} sequences")

In [ ]:
# PyTorch Dataset class
class TFTDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        
        return {
            'household_id': torch.LongTensor([seq['household_id']]),
            'encoder_hour': torch.LongTensor(seq['encoder_hour']),
            'encoder_dow': torch.LongTensor(seq['encoder_dow']),
            'encoder_weekend': torch.FloatTensor(seq['encoder_weekend']),
            'encoder_temp': torch.FloatTensor(seq['encoder_temp']),
            'encoder_consumption': torch.FloatTensor(seq['encoder_consumption']),
            'decoder_hour': torch.LongTensor(seq['decoder_hour']),
            'decoder_dow': torch.LongTensor(seq['decoder_dow']),
            'decoder_weekend': torch.FloatTensor(seq['decoder_weekend']),
            'target': torch.FloatTensor(seq['target'])
        }

# Create DataLoaders
batch_size = 64

train_dataset = TFTDataset(train_sequences)
val_dataset = TFTDataset(val_sequences)
test_dataset = TFTDataset(test_sequences)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\n✅ DataLoaders created (batch_size={batch_size})")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

---

## 🏗️ Часть 3: TFT Building Blocks

### 3.1 Gated Residual Network (GRN)

**Key component TFT!**

**Архитектура:**
```
Input x
  ↓
ELU → Linear → Dropout  ← context (optional)
  ↓
ELU → Linear → Dropout
  ↓
GLU (Gated Linear Unit)
  ↓
LayerNorm(x + output)  ← Residual
```

**Зачем:**
- Gating mechanism для feature selection
- Residual connections для gradient flow
- Context injection (например, static features)


In [ ]:
class GatedResidualNetwork(nn.Module):
    """
    Gated Residual Network (GRN)
    
    Key building block of TFT.
    Applies gating mechanism with residual connections.
    """
    def __init__(self, input_size, hidden_size, output_size, 
                 dropout=0.1, context_size=None):
        super(GatedResidualNetwork, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.context_size = context_size
        
        # Linear layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        
        if context_size is not None:
            self.context_fc = nn.Linear(context_size, hidden_size, bias=False)
        
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
        # Gated Linear Unit
        self.gate_fc = nn.Linear(hidden_size, output_size)
        
        # Skip connection
        if input_size != output_size:
            self.skip_fc = nn.Linear(input_size, output_size)
        else:
            self.skip_fc = None
        
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(output_size)
    
    def forward(self, x, context=None):
        """
        Args:
            x: (batch, ..., input_size)
            context: optional (batch, ..., context_size)
        
        Returns:
            output: (batch, ..., output_size)
        """
        # Feed-forward
        hidden = F.elu(self.fc1(x))
        
        # Context injection
        if context is not None and self.context_size is not None:
            hidden = hidden + self.context_fc(context)
        
        hidden = self.dropout(hidden)
        hidden = F.elu(self.fc2(hidden))
        hidden = self.dropout(hidden)
        
        # Gating (GLU - Gated Linear Unit)
        gate = torch.sigmoid(self.gate_fc(hidden))
        
        # Skip connection
        if self.skip_fc is not None:
            x = self.skip_fc(x)
        
        # Residual + LayerNorm
        output = self.layer_norm(x + gate * hidden)
        
        return output

print("✅ GatedResidualNetwork implemented")

### 3.2 Variable Selection Network

**Automatic feature selection!**

**Проблема:** Не все features важны для каждого prediction

**Решение:**
1. Transform каждую feature через GRN
2. Compute variable weights (softmax)
3. Weighted combination

**Формула:**
$$\text{weights} = \text{softmax}(\text{GRN}([\text{flatten}(\text{features})]))$$
$$\text{output} = \sum_i \text{weights}_i \cdot \text{GRN}(\text{feature}_i)$$

In [ ]:
class VariableSelectionNetwork(nn.Module):
    """
    Variable Selection Network
    
    Automatically selects relevant features.
    """
    def __init__(self, input_sizes, hidden_size, output_size, dropout=0.1):
        """
        Args:
            input_sizes: list of input sizes for each variable
            hidden_size: hidden dimension
            output_size: output dimension
        """
        super(VariableSelectionNetwork, self).__init__()
        
        self.num_vars = len(input_sizes)
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # GRN для каждой variable
        self.variable_grns = nn.ModuleList([
            GatedResidualNetwork(input_size, hidden_size, output_size, dropout)
            for input_size in input_sizes
        ])
        
        # GRN для variable selection weights
        total_input_size = sum(input_sizes)
        self.weight_grn = GatedResidualNetwork(
            total_input_size, hidden_size, self.num_vars, dropout
        )
    
    def forward(self, variables, return_weights=False):
        """
        Args:
            variables: list of tensors, each (batch, ..., input_size_i)
            return_weights: whether to return variable importance weights
        
        Returns:
            output: (batch, ..., output_size)
            weights: optional (batch, num_vars)
        """
        # Transform each variable через GRN
        transformed = [grn(var) for grn, var in zip(self.variable_grns, variables)]
        
        # Concatenate для weight computation
        flattened = torch.cat([var.flatten(start_dim=1) for var in variables], dim=-1)
        
        # Compute variable weights
        weights = self.weight_grn(flattened)  # (batch, num_vars)
        weights = F.softmax(weights, dim=-1)  # normalize
        
        # Weighted sum
        # Stack: (batch, num_vars, ..., output_size)
        stacked = torch.stack(transformed, dim=1)
        
        # Expand weights для broadcasting
        weights_expanded = weights.unsqueeze(-1).unsqueeze(-1)  # (batch, num_vars, 1, 1)
        
        # Weighted combination
        output = (stacked * weights_expanded).sum(dim=1)  # (batch, ..., output_size)
        
        if return_weights:
            return output, weights
        return output

print("✅ VariableSelectionNetwork implemented")

### 3.3 Multi-Head Attention (from Phase 4 Step 1)

Переиспользуем компоненты из Step 1!

In [ ]:
class ScaledDotProductAttention(nn.Module):
    """Scaled Dot-Product Attention"""
    def __init__(self, dropout=0.1):
        super(ScaledDotProductAttention, self).__init__()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, Q, K, V, mask=None):
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        context = torch.matmul(attention_weights, V)
        return context, attention_weights


class MultiHeadAttention(nn.Module):
    """Multi-Head Attention"""
    def __init__(self, d_model, n_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % n_heads == 0
        
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        
        self.attention = ScaledDotProductAttention(dropout)
        self.W_O = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def split_heads(self, x):
        batch_size, seq_len, d_model = x.size()
        return x.view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
    
    def combine_heads(self, x):
        batch_size, n_heads, seq_len, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
    
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_Q(Q))
        K = self.split_heads(self.W_K(K))
        V = self.split_heads(self.W_V(V))
        
        context, attention_weights = self.attention(Q, K, V, mask)
        context = self.combine_heads(context)
        output = self.W_O(context)
        output = self.dropout(output)
        
        return output, attention_weights

print("✅ Multi-Head Attention loaded (from Phase 4 Step 1)")

### 3.4 Test Building Blocks

In [ ]:
# Test GRN
print("Testing GatedResidualNetwork...")
grn = GatedResidualNetwork(input_size=10, hidden_size=32, output_size=16)
x_test = torch.randn(4, 5, 10)  # (batch=4, seq=5, features=10)
out = grn(x_test)
print(f"  Input: {x_test.shape} → Output: {out.shape}")
assert out.shape == (4, 5, 16), "GRN output shape mismatch"
print("  ✅ GRN works!")

# Test VSN
print("\nTesting VariableSelectionNetwork...")
vsn = VariableSelectionNetwork(input_sizes=[10, 15, 20], hidden_size=32, output_size=16)
vars_test = [
    torch.randn(4, 5, 10),
    torch.randn(4, 5, 15),
    torch.randn(4, 5, 20)
]
out, weights = vsn(vars_test, return_weights=True)
print(f"  Output: {out.shape}")
print(f"  Weights: {weights.shape}")
print(f"  Weights sum: {weights[0].sum().item():.4f} (should be 1.0)")
assert out.shape == (4, 5, 16), "VSN output shape mismatch"
assert weights.shape == (4, 3), "VSN weights shape mismatch"
print("  ✅ VSN works!")

# Test MHA
print("\nTesting MultiHeadAttention...")
mha = MultiHeadAttention(d_model=64, n_heads=4)
x_test = torch.randn(4, 10, 64)  # (batch=4, seq=10, d_model=64)
out, attn = mha(x_test, x_test, x_test)
print(f"  Input: {x_test.shape} → Output: {out.shape}")
print(f"  Attention weights: {attn.shape}")
assert out.shape == (4, 10, 64), "MHA output shape mismatch"
print("  ✅ MHA works!")

print("\n" + "="*60)
print("✅ All TFT building blocks implemented and tested!")
print("="*60)

---

## 🏗️ Часть 4: Simplified Temporal Fusion Transformer

### 4.1 Full TFT Architecture

**Упрощенная версия TFT для демонстрации key concepts:**

1. **Input Processing**: Embeddings для categorical, linear для numerical
2. **Variable Selection**: Автоматический выбор важных features
3. **LSTM Encoder**: Process historical sequence
4. **Multi-Head Attention**: Long-range dependencies
5. **Decoder**: Multi-horizon predictions (quantile forecasts)

**Simplifications (для обучения):**
- Simplified static covariate enrichment
- Single LSTM layer (вместо многослойного)
- Simplified temporal fusion

**Сохраняем ключевое:**
- Multi-horizon forecasting
- Variable selection
- Attention mechanism
- Quantile predictions

In [ ]:
class SimplifiedTFT(nn.Module):
    """
    Simplified Temporal Fusion Transformer
    
    Key components:
    - Variable Selection Networks
    - LSTM Encoder
    - Multi-Head Attention
    - Multi-horizon Quantile Output
    """
    def __init__(self,
                 num_households,
                 encoder_length,
                 decoder_length,
                 hidden_size=64,
                 n_heads=4,
                 n_quantiles=3,
                 dropout=0.1):
        super(SimplifiedTFT, self).__init__()
        
        self.encoder_length = encoder_length
        self.decoder_length = decoder_length
        self.hidden_size = hidden_size
        self.n_quantiles = n_quantiles
        
        # 1. Embeddings
        self.household_embedding = nn.Embedding(num_households, 8)
        self.hour_embedding = nn.Embedding(24, 8)
        self.dow_embedding = nn.Embedding(7, 4)
        
        # 2. Variable Selection для encoder inputs
        # Features: hour_emb(8) + dow_emb(4) + weekend(1) + temp(1) + consumption(1)
        encoder_input_sizes = [8, 4, 1, 1, 1]  # 5 variables
        self.encoder_vsn = VariableSelectionNetwork(
            encoder_input_sizes, hidden_size, hidden_size, dropout
        )
        
        # 3. LSTM Encoder
        self.lstm_encoder = nn.LSTM(
            hidden_size, hidden_size, batch_first=True, dropout=dropout
        )
        
        # 4. Multi-Head Attention
        self.attention = MultiHeadAttention(hidden_size, n_heads, dropout)
        
        # 5. GRN после attention
        self.post_attention_grn = GatedResidualNetwork(
            hidden_size, hidden_size, hidden_size, dropout
        )
        
        # 6. Decoder для future inputs
        decoder_input_sizes = [8, 4, 1]  # hour_emb + dow_emb + weekend
        self.decoder_vsn = VariableSelectionNetwork(
            decoder_input_sizes, hidden_size, hidden_size, dropout
        )
        
        # 7. LSTM Decoder
        self.lstm_decoder = nn.LSTM(
            hidden_size, hidden_size, batch_first=True, dropout=dropout
        )
        
        # 8. Quantile output layer
        self.output_layer = nn.Linear(hidden_size, n_quantiles)
    
    def forward(self, batch, return_attention=False):
        """
        Args:
            batch: dict with encoder/decoder inputs
            return_attention: whether to return attention weights
        
        Returns:
            quantiles: (batch, decoder_length, n_quantiles)
            attention_weights: optional
        """
        batch_size = batch['encoder_hour'].size(0)
        
        # 1. Process encoder inputs
        # Embeddings
        encoder_hour_emb = self.hour_embedding(batch['encoder_hour'])  # (B, enc_len, 8)
        encoder_dow_emb = self.dow_embedding(batch['encoder_dow'])     # (B, enc_len, 4)
        encoder_weekend = batch['encoder_weekend'].unsqueeze(-1)       # (B, enc_len, 1)
        encoder_temp = batch['encoder_temp'].unsqueeze(-1)             # (B, enc_len, 1)
        encoder_consumption = batch['encoder_consumption'].unsqueeze(-1)  # (B, enc_len, 1)
        
        # Variable selection
        encoder_variables = [
            encoder_hour_emb,
            encoder_dow_emb,
            encoder_weekend,
            encoder_temp,
            encoder_consumption
        ]
        encoder_selected = self.encoder_vsn(encoder_variables)  # (B, enc_len, hidden)
        
        # 2. LSTM Encoder
        encoder_output, (h_n, c_n) = self.lstm_encoder(encoder_selected)
        # encoder_output: (B, enc_len, hidden)
        
        # 3. Multi-Head Attention
        attn_output, attention_weights = self.attention(
            encoder_output, encoder_output, encoder_output
        )
        
        # 4. GRN после attention
        enriched = self.post_attention_grn(attn_output)
        # enriched: (B, enc_len, hidden)
        
        # 5. Process decoder inputs (future known)
        decoder_hour_emb = self.hour_embedding(batch['decoder_hour'])  # (B, dec_len, 8)
        decoder_dow_emb = self.dow_embedding(batch['decoder_dow'])     # (B, dec_len, 4)
        decoder_weekend = batch['decoder_weekend'].unsqueeze(-1)       # (B, dec_len, 1)
        
        decoder_variables = [
            decoder_hour_emb,
            decoder_dow_emb,
            decoder_weekend
        ]
        decoder_selected = self.decoder_vsn(decoder_variables)  # (B, dec_len, hidden)
        
        # 6. LSTM Decoder (с context от encoder)
        decoder_output, _ = self.lstm_decoder(decoder_selected, (h_n, c_n))
        # decoder_output: (B, dec_len, hidden)
        
        # 7. Quantile predictions
        quantiles = self.output_layer(decoder_output)  # (B, dec_len, n_quantiles)
        
        if return_attention:
            return quantiles, attention_weights
        return quantiles

print("✅ SimplifiedTFT implemented")

In [ ]:
# Initialize model
model = SimplifiedTFT(
    num_households=10,
    encoder_length=ENCODER_LENGTH,
    decoder_length=DECODER_LENGTH,
    hidden_size=64,
    n_heads=4,
    n_quantiles=3,  # P10, P50, P90
    dropout=0.1
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Simplified TFT Model:")
print("="*60)
print(f"  Encoder length: {ENCODER_LENGTH}")
print(f"  Decoder length: {DECODER_LENGTH}")
print(f"  Hidden size: 64")
print(f"  Attention heads: 4")
print(f"  Quantiles: 3 (P10, P50, P90)")
print(f"\nModel size:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print("="*60)

### 4.2 Quantile Loss

**TFT предсказывает quantiles для uncertainty estimation:**

- **P10 (10th percentile)**: Optimistic forecast
- **P50 (median)**: Most likely forecast
- **P90 (90th percentile)**: Pessimistic forecast

**Quantile Loss:**

$$L_q(y, \hat{y}) = \begin{cases}
q \cdot (y - \hat{y}) & \text{if } y \geq \hat{y} \\
(1-q) \cdot (\hat{y} - y) & \text{if } y < \hat{y}
\end{cases}$$

Где $q$ - quantile level (0.1, 0.5, 0.9)

In [ ]:
class QuantileLoss(nn.Module):
    """
    Quantile Loss for probabilistic forecasting
    """
    def __init__(self, quantiles=[0.1, 0.5, 0.9]):
        super(QuantileLoss, self).__init__()
        self.quantiles = torch.FloatTensor(quantiles)
    
    def forward(self, predictions, targets):
        """
        Args:
            predictions: (batch, time, n_quantiles)
            targets: (batch, time)
        
        Returns:
            loss: scalar
        """
        self.quantiles = self.quantiles.to(predictions.device)
        
        targets = targets.unsqueeze(-1)  # (batch, time, 1)
        errors = targets - predictions    # (batch, time, n_quantiles)
        
        quantiles_expanded = self.quantiles.view(1, 1, -1)
        
        loss = torch.max(
            quantiles_expanded * errors,
            (quantiles_expanded - 1) * errors
        )
        
        return loss.mean()

# Loss and optimizer
criterion = QuantileLoss(quantiles=[0.1, 0.5, 0.9])
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

print("✅ QuantileLoss и optimizer initialized")

### 4.3 Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in loader:
        # Move to device
        batch = {k: v.to(device) for k, v in batch.items()}
        
        optimizer.zero_grad()
        
        # Forward
        quantiles = model(batch)
        loss = criterion(quantiles, batch['target'])
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item() * batch['target'].size(0)
    
    avg_loss = total_loss / len(loader.dataset)
    return avg_loss


def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            quantiles = model(batch)
            loss = criterion(quantiles, batch['target'])
            total_loss += loss.item() * batch['target'].size(0)
    
    avg_loss = total_loss / len(loader.dataset)
    return avg_loss

print("✅ Training functions defined")

In [ ]:
# Training loop
num_epochs = 20
history = {'train_loss': [], 'val_loss': []}

print(f"Training TFT for {num_epochs} epochs...")
print(f"Dataset: {len(train_dataset):,} train, {len(val_dataset):,} val")
print("="*60)

best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    
    scheduler.step(val_loss)
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        # Save best model
        torch.save(model.state_dict(), '/tmp/best_tft.pt')
    
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1:2d}/{num_epochs}] "
              f"Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")

print("\n" + "="*60)
print(f"✅ Training completed!")
print(f"Best Val Loss: {best_val_loss:.4f}")

# Load best model
model.load_state_dict(torch.load('/tmp/best_tft.pt'))
print("✅ Best model loaded")

In [ ]:
# Plot training history
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss', linewidth=2)
plt.plot(history['val_loss'], label='Val Loss', linewidth=2)
plt.axhline(y=best_val_loss, color='red', linestyle='--', 
           label=f'Best: {best_val_loss:.4f}', linewidth=1.5)
plt.title('Training History', fontsize=16, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Quantile Loss', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 4.4 Evaluation и Visualization

In [ ]:
# Evaluate на test set
test_loss = evaluate(model, test_loader, criterion, device)
print(f"Test Quantile Loss: {test_loss:.4f}")

# Get predictions для нескольких примеров
model.eval()
sample_predictions = []
sample_targets = []
sample_encoders = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        if i >= 3:  # first 3 batches
            break
        
        batch = {k: v.to(device) for k, v in batch.items()}
        quantiles = model(batch)
        
        sample_predictions.append(quantiles.cpu().numpy())
        sample_targets.append(batch['target'].cpu().numpy())
        sample_encoders.append(batch['encoder_consumption'].cpu().numpy())

sample_predictions = np.concatenate(sample_predictions, axis=0)
sample_targets = np.concatenate(sample_targets, axis=0)
sample_encoders = np.concatenate(sample_encoders, axis=0)

print(f"\nSample predictions shape: {sample_predictions.shape}")
print(f"  {sample_predictions.shape[0]} samples × {sample_predictions.shape[1]} horizons × {sample_predictions.shape[2]} quantiles")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

for idx in range(3):
    ax = axes[idx]
    
    # Historical
    historical = sample_encoders[idx]
    historical_x = np.arange(-ENCODER_LENGTH, 0)
    
    # Future (target)
    target = sample_targets[idx]
    future_x = np.arange(0, DECODER_LENGTH)
    
    # Predictions (P10, P50, P90)
    pred_p10 = sample_predictions[idx, :, 0]
    pred_p50 = sample_predictions[idx, :, 1]
    pred_p90 = sample_predictions[idx, :, 2]
    
    # Plot
    ax.plot(historical_x, historical, 'k-', linewidth=2, label='Historical')
    ax.plot(future_x, target, 'b-', linewidth=2, marker='o', 
           markersize=4, label='Actual (Target)')
    ax.plot(future_x, pred_p50, 'r-', linewidth=2, marker='s', 
           markersize=4, label='Prediction (P50)')
    
    # Uncertainty bands
    ax.fill_between(future_x, pred_p10, pred_p90, alpha=0.2, 
                    color='red', label='P10-P90 Range')
    
    ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
    ax.set_title(f'Sample {idx+1}: Multi-Horizon Forecast ({DECODER_LENGTH}h ahead)', 
                fontsize=14, fontweight='bold')
    ax.set_xlabel('Time (hours)', fontsize=11)
    ax.set_ylabel('Consumption (kWh)', fontsize=11)
    ax.legend(loc='best')
    ax.grid(alpha=0.3)

plt.suptitle('TFT Multi-Horizon Probabilistic Forecasts', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("  - P50 (median) follows the actual trend")
print("  - P10-P90 range captures uncertainty")
print("  - Wider uncertainty band for longer horizons (expected!)")
print("  - Model learns daily patterns from historical data")

### 4.5 Quantitative Metrics

In [ ]:
# Compute metrics для P50 (median forecast)
p50_predictions = sample_predictions[:, :, 1]  # P50

# Overall metrics
mae = mean_absolute_error(sample_targets.flatten(), p50_predictions.flatten())
rmse = np.sqrt(mean_squared_error(sample_targets.flatten(), p50_predictions.flatten()))

print("TFT Performance (P50):")
print("="*60)
print(f"  MAE (Mean Absolute Error): {mae:.4f} kWh")
print(f"  RMSE (Root Mean Squared Error): {rmse:.4f} kWh")
print("="*60)

# Per-horizon metrics
horizon_mae = []
for h in range(DECODER_LENGTH):
    h_mae = mean_absolute_error(sample_targets[:, h], p50_predictions[:, h])
    horizon_mae.append(h_mae)

# Plot horizon performance
plt.figure(figsize=(12, 5))
plt.plot(range(1, DECODER_LENGTH+1), horizon_mae, linewidth=2, marker='o')
plt.title('MAE by Forecast Horizon', fontsize=16, fontweight='bold')
plt.xlabel('Forecast Horizon (hours)', fontsize=12)
plt.ylabel('MAE (kWh)', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 MAE increases with horizon (expected!):")
print(f"  Hour 1: {horizon_mae[0]:.4f} kWh")
print(f"  Hour 12: {horizon_mae[11]:.4f} kWh")
print(f"  Hour 24: {horizon_mae[23]:.4f} kWh")

### 4.6 Attention Weights Visualization

**Interpretability через attention:**
- Какие historical timesteps важны для каждого forecast horizon?
- Long-range vs short-range dependencies

In [ ]:
# Get attention weights для одного примера
model.eval()
with torch.no_grad():
    sample_batch = {k: v[:1].to(device) for k, v in next(iter(test_loader)).items()}
    _, attention_weights = model(sample_batch, return_attention=True)

# attention_weights: (batch, n_heads, encoder_len, encoder_len)
# Average over heads
attn = attention_weights[0].mean(dim=0).cpu().numpy()  # (encoder_len, encoder_len)

print(f"Attention weights shape: {attn.shape}")

# Visualize
plt.figure(figsize=(12, 10))
sns.heatmap(attn, cmap='YlOrRd', cbar_kws={'label': 'Attention Weight'})
plt.title('TFT Attention Weights (Averaged over Heads)', 
         fontsize=16, fontweight='bold')
plt.xlabel('Keys (Historical Timesteps)', fontsize=12)
plt.ylabel('Queries (Historical Timesteps)', fontsize=12)
plt.tight_layout()
plt.show()

print("\n📊 Attention interpretation:")
print("  - Diagonal: current timestep attends to itself (expected)")
print("  - Patterns: model learns temporal dependencies")
print("  - Nearby timesteps often have higher attention (local patterns)")
print("  - Some long-range dependencies visible (weekly seasonality)")

---

## 🎓 Итоги и Выводы

### Что мы реализовали

#### 1. Temporal Fusion Transformer

**Ключевые компоненты:**
- ✅ **Variable Selection Networks**: автоматический выбор важных features
- ✅ **Gated Residual Networks**: гибкие building blocks с gating
- ✅ **LSTM Encoder**: обработка historical sequence
- ✅ **Multi-Head Attention**: long-range temporal dependencies
- ✅ **Multi-Horizon Output**: одновременное предсказание 24 часов
- ✅ **Quantile Forecasting**: uncertainty estimation (P10, P50, P90)

**Dataset:**
- 87,600 data points (10 households × 365 days × 24 hours)
- Mixed features: categorical (household, hour, day) + numerical (temp)
- Real-world patterns: daily + weekly seasonality, temperature dependency

---

### Ключевые достижения

#### 1. Multi-Horizon Forecasting

**Традиционные подходы:**
- Recursive: предсказать t+1, использовать для t+2, ... (накопление ошибок!)
- Direct: отдельные модели для каждого horizon (дорого!)

**TFT:**
- ✅ Single model predicts all 24 horizons simultaneously
- ✅ No error accumulation
- ✅ Shared representations между horizons

---

#### 2. Probabilistic Forecasts

**Point forecasts (обычные модели):**
- Только P50 (median)
- Нет uncertainty информации

**TFT Quantile Forecasts:**
- ✅ P10, P50, P90 quantiles
- ✅ Uncertainty bands (P10-P90 range)
- ✅ Risk assessment (важно для decision making)

**Пример использования:**
```
Electricity demand planning:
- P90: pessimistic case → reserve capacity
- P50: expected case → normal operations
- P10: optimistic case → minimum requirements
```

---

#### 3. Interpretability

**Attention Weights:**
- Показывают, какие historical timesteps важны
- Можно объяснить каждое предсказание

**Variable Selection:**
- Автоматически выбирает важные features
- Можно проанализировать, какие features наиболее полезны

---

### Объединение Phase 3 + Phase 4

**TFT combines best of both worlds:**

| Component | From Phase | Purpose |
|-----------|-----------|----------|
| LSTM Encoder | Phase 3 (RNN) | Sequential processing |
| Multi-Head Attention | Phase 4 (Transformers) | Long-range dependencies |
| Variable Selection | Phase 4 (TabTransformer) | Automatic feature engineering |
| Time Series Patterns | Phase 3 | Seasonality, trends |

**Результат:**
- LSTM handles sequential nature
- Attention captures long-term patterns
- Variable selection handles mixed features
- Quantiles provide uncertainty

---

### Когда использовать TFT?

**✅ Идеально подходит когда:**
1. **Multi-horizon forecasting** (нужно предсказать несколько шагов)
2. **Mixed data types** (categorical + numerical + temporal)
3. **Uncertainty важна** (risk assessment, decision making)
4. **Interpretability нужна** (объяснить predictions stakeholders)
5. **Multiple time series** (panel data, hundreds/thousands series)
6. **Complex patterns** (multiple seasonalities, interactions)

**❌ Может быть overkill когда:**
1. Simple univariate time series (Prophet может быть достаточно)
2. Single-step forecasting (LSTM может быть проще)
3. Small dataset (<1000 timesteps)
4. Только point forecasts нужны (не quantiles)

---

### Real-World Applications

**TFT используется в:**

1. **Electricity Load Forecasting**
   - Multi-step ahead predictions
   - Uncertainty для capacity planning
   - Multiple households/regions

2. **Retail Demand Forecasting**
   - SKU-level predictions
   - Promotional events (categorical)
   - Quantiles для inventory optimization

3. **Traffic Prediction**
   - Multiple roads/intersections
   - Weather, events влияют
   - Uncertainty для routing

4. **Financial Forecasting**
   - Multi-asset predictions
   - Risk assessment (VaR через quantiles)
   - Market conditions (categorical)

---

## 🎉 Поздравляем!

Вы завершили **Phase 4: Transformers & Modern Architectures**!

**Теперь вы знаете:**
- 🧠 Как работает Self-Attention mechanism
- 🔄 Как Transformers превосходят RNN для parallelization
- 📊 Как применять Transformers к табличным данным (TabTransformer)
- ⏰ Как использовать TFT для multi-horizon time series forecasting
- 📈 Как получать probabilistic forecasts (quantiles)
- 🔍 Как интерпретировать модели через attention

**Путешествие через ML Roadmap:**
- ✅ Phase 1: Classical ML (XGBoost, LightGBM, CatBoost, Stacking)
- ✅ Phase 2: Deep Learning Basics (MLP, CNN, Autoencoders)
- ✅ Phase 3: Temporal Data & RNN (ARIMA, LSTM, Attention)
- ✅ Phase 4: Transformers (Self-Attention, TabTransformer, TFT)

**Следующий шаг:** Transfer Learning, Pre-training, Foundation Models! 🚀
